In [3]:
import doekbase.data_api
from doekbase.data_api.annotation.genome_annotation.api import GenomeAnnotationAPI
import time
import os

In [8]:
def get_gff(gene,genome_annotation, taxon_id, source_info):
    
    print(gene)
    #retrieve pairwise inter-feature relationships between genes, mRNAs, CDSs
    #the below section of code has method timings torecord performance  
    t0 = time.time()
    listmrna=genome_annotation.get_mrna_by_gene([gene])[gene]    
    t1 = time.time()
    print "get_mrna_by_gene " , (t1-t0), "s"
    #print(listmrna)
    
    #t0 = time.time()
    #listcds=genome_annotation.get_cds_by_gene([gene])[gene] 
    #t1 = time.time()
    #print "get_cds_by_gene " , (t1-t0), "s"   
    #print(listcds)
        
    if len(listmrna) > 0:
        t0 = time.time()
        print(listmrna)
        mrna_cds=genome_annotation.get_cds_by_mrna(listmrna)
        t1 = time.time()
        print "get_cds_by_mrna " , (t1-t0), "s"
        #print(mrna_cds)
    
        #t0 = time.time()
        #cds_mrna=genome_annotation.get_mrna_by_cds(listcds)
        #t1 = time.time()
        #print "get_mrna_by_cds " , (t1-t0), "s"

        t0 = time.time()
        get_mrna_exons = genome_annotation.get_mrna_exons(listmrna)
        t1 = time.time()
        print "get_mrna_exons " , (t1-t0), "s"
        #print get_mrna_exons
        
        #retrieve locations for mRNAs
        t0 = time.time()
        mrna_location=genome_annotation.get_feature_locations(listmrna) 
        t1 = time.time()
        print "get_feature_locations " , (t1-t0), "s"


    #retrieve locations for genes, mRNAs, and cds
    t0 = time.time()
    gene_location=genome_annotation.get_feature_locations([gene]) 
    t1 = time.time()
    print "get_feature_locations " , (t1-t0), "s"
    
    #t0 = time.time()
    #cds_location=genome_annotation.get_feature_locations(listcds) 
    #t1 = time.time()
    #print "get_feature_locations " , (t1-t0), "s"
    
    string=""
    
    string+="##sequence-region "+gene_location[gene_location.keys()[0]][0]['contig_id']+"\t"+"start"+"\t"+"stop"+"\n"##NC_003070.9 1 30427671
    if taxon_id != -1:
        string+="##species http://www.ncbi.nlm.nih.gov/Taxonomy/Browser/wwwtax.cgi?id="+str(taxon_id)+"\n"#http://www.ncbi.nlm.nih.gov/Taxonomy/Browser/wwwtax.cgi?id=3702
    else :
        string+="##species unknown\n"
    

    contig_id = ""
    #output the gene data
    for x in gene_location:
        #print(type(x))
        #print(x)
        info=gene_location[x][0]
        #print(type(info))
        #print info
        stop=info['start'] + info['length']
        
        contig_id = info['contig_id']
        
        #print "source_info"
        #print(type(source_info))
        #print(source_info)
        #output aliases for gene as dbxref?
        string+=info['contig_id']  + "\t" + source_info['external_source']+"\tgene\t"  + str(info['start']) + "\t" + str(stop) + "\t.\t" +  info['strand']+ "\t.\t" + "ID=" + x + ";" + "Name="+ x +"\n"
    
    print "len(listmrna)"    
    print(len(listmrna))
    if len(listmrna) > 0:    
    #output the mRNA data
        for mrna in listmrna:
            print("mrna "+mrna)
            cds=mrna_cds[mrna]
            infomrna=mrna_location[mrna][0]
            #print(type(infomrna))
            #print(infomrna)
            #infocds=cds_location[mrna_cds[mrna]]
            stop=infomrna['start'] + infomrna['length']            
            ###output mrna data
            string+=infomrna['contig_id']  + "\t" +source_info['external_source']+"\tmRNA\t"  + str(infomrna['start'])+ "\t" + str(stop) + "\t.\t" +  infomrna['strand']+ "\t.\t" + "ID=" + mrna + ";" + "Parent="+ gene+"\n"
            
            #print 'get_mrna_exons.keys()'
            #print get_mrna_exons.keys()
            
            ##TODO output UTRs
            for mrna in get_mrna_exons.keys():
                #print(type(get_mrna_exons[mrna]))
                #print(get_mrna_exons[mrna])
                #print "get_mrna_exons[mrna][i]['exon_location']"
                #print type(get_mrna_exons[mrna][0]['exon_location'])
                ###output exon data
                for i in range(0, len(get_mrna_exons[mrna])):
                    #print "mrna, gene"
                    #print type(mrna)
                    #print type(gene)
                    stop= get_mrna_exons[mrna][i]['exon_location']['start'] + get_mrna_exons[mrna][i]['exon_location']['length']
                    string+= get_mrna_exons[mrna][i]['exon_location']['contig_id']  + "\t" +source_info['external_source'] + "\texon\t"  + str(get_mrna_exons[mrna][i]['exon_location']['start'])+ "\t" + str(stop) + "\t.\t" +  get_mrna_exons[mrna][i]['exon_location']['strand']+ "\t.\t" + "ID=" + mrna +".exon." + str(i)  + ";" + "Parent="+ mrna+"\n"
                ###output CDS data
                for i in range(0, len(get_mrna_exons[mrna])):
                    stop= get_mrna_exons[mrna][i]['exon_location']['start'] + get_mrna_exons[mrna][i]['exon_location']['length']
                    ##***TODO: set PHASE for CDS
                    string+= get_mrna_exons[mrna][i]['exon_location']['contig_id']  + "\t" +source_info['external_source'] + "\tCDS\t"  + str(get_mrna_exons[mrna][i]['exon_location']['start'])+ "\t" + str(stop) + "\t.\t" +  get_mrna_exons[mrna][i]['exon_location']['strand']+ "\t.\t" + "ID=" + mrna +".CDS." + str(i)  + ";" + "Parent="+ mrna+"\n"

    return string       

#hardcoded test with 5 random genes
def runDemo(ws_url='https://ci.kbase.us/services/ws/', ws="ReferenceEnsemblPlantGenomeAnnotations", genome_id="280699_Ensembl"):
    #an example KBase reference genome
    genome_ref = "ReferenceEnsemblPlantGenomeAnnotations"+"/"+genome_id
    #instantiate a new genome annotation API
    genome_annotation = GenomeAnnotationAPI(services = {"workspace_service_url": ws_url}, token=os.environ.get('KB_AUTH_TOKEN'), ref=genome_ref)
    taxon = genome_annotation.get_taxon()
    taxonid = taxon.get_taxonomic_id()
    
    if taxonid == -1 :
        taxonid = "unknown"
    
    assembly = genome_annotation.get_assembly()
    contig_ids = assembly.get_contig_ids()
    sourceinfo = assembly.get_external_source_info()
    genes=['gene_1','gene_2','gene_5', 'gene_100', 'gene_1000']
    #genes=['gene_1']
    
    gffdata=""
    gffdata+="##gff-version 3\n"
    gffdata+="#!gff-spec-version 1.21\n"
    gffdata+="#!processor KBase GFF3 downloader\n"
    #!genome-build TAIR10
    #!genome-build-accession NCBI_Assembly:GCF_000001735.3   
    
    for s in genes:
        gffdata+=get_gff(s,genome_annotation,taxonid, sourceinfo)

    gffdata+="###"
    outfile = genome_id+'.gff'
    print outfile
    with open(outfile, 'w') as f:
        f.write(gffdata)
    
#create GFF for an entire GenomeAnnotation
def runGenome(ws_url='https://ci.kbase.us/services/ws/', ws="ReferenceEnsemblPlantGenomeAnnotations", genome_id="280699_Ensembl"):
    #an example KBase reference genome
    genome_ref = "ReferenceEnsemblPlantGenomeAnnotations"+"/"+genome_id#kb|g.166819"
    #instantiate a new genome annotation API
    genome_annotation = GenomeAnnotationAPI(services = {"workspace_service_url": ws_url}, token=os.environ.get('KB_AUTH_TOKEN'), ref=genome_ref)
    taxon = genome_annotation.get_taxon()
    taxonid = taxon.get_taxonomic_id()
    
    if taxonid == -1 :
        taxonid = "unknown"
    
    assembly = genome_annotation.get_assembly()
    contig_ids = assembly.get_contig_ids()
    sourceinfo = assembly.get_external_source_info()
    
    gffdata=""
    gffdata+="##gff-version 3\n"
    gffdata+="#!gff-spec-version 1.21\n"
    gffdata+="#!processor KBase GFF3 downloader\n"
    #!genome-build TAIR10
    #!genome-build-accession NCBI_Assembly:GCF_000001735.3  
    
    start=1
    stop=100000000
    
    feat_data = []
    print "contig_ids len "+ str(len(contig_ids))
    for c in contig_ids:
        regions = []
        contig_add = {"contig_id": c, "start": start, "length": stop-start, "strand": "+"}
        contig_add2 = {"contig_id": c, "start": stop, "length": stop-start, "strand": "-"}
        regions.append(contig_add)
        regions.append(contig_add2)
        feat_data = genome_annotation.get_feature_ids(filters={"region_list":regions,"type_list":["gene"]})['by_type']['gene']
        
        print "feat_data len "+ str(len(feat_data))
        #print feat_data
        #for s in genes:
        for f in feat_data:
            print f
            gffdata+=get_gff(f,genome_annotation,taxonid, sourceinfo)
            
        #break after first contig for now    
        break;

    gffdata+="###"
    outfile = genome_id+'.gff'
    print outfile
    with open(outfile, 'w') as f:
        f.write(gffdata)
        
    feat_data

In [7]:
genomeid="280699_Ensembl"
wsthis = "ReferenceEnsemblPlantGenomeAnnotations"
run(ws=wsthis, genome_id=genomeid)

gene_1
get_mrna_by_gene  1.67064189911 s
[u'mRNA_1']
get_cds_by_mrna  1.50494503975 s
get_mrna_exons  1.49544405937 s
get_feature_locations  1.95886707306 s
get_feature_locations  2.66089892387 s
len(listmrna)
1
mrna mRNA_1
gene_2
get_mrna_by_gene  1.5816628933 s
[u'mRNA_2']
get_cds_by_mrna  1.58651709557 s
get_mrna_exons  1.58610486984 s
get_feature_locations  1.94034600258 s
get_feature_locations  1.88635587692 s
len(listmrna)
1
mrna mRNA_2
gene_5
get_mrna_by_gene  1.54612207413 s
[u'mRNA_3']
get_cds_by_mrna  1.53069210052 s
get_mrna_exons  1.67014789581 s
get_feature_locations  2.21215605736 s
get_feature_locations  2.13090801239 s
len(listmrna)
1
mrna mRNA_3
gene_100
get_mrna_by_gene  1.62239289284 s
[u'mRNA_63']
get_cds_by_mrna  1.63559103012 s
get_mrna_exons  1.58180403709 s
get_feature_locations  1.98508691788 s
get_feature_locations  2.41667795181 s
len(listmrna)
1
mrna mRNA_63
gene_1000
get_mrna_by_gene  1.99698400497 s
[u'mRNA_932']
get_cds_by_mrna  1.54204797745 s
get_mrna_e

In [9]:
genomeid="280699_Ensembl"
wsthis = "ReferenceEnsemblPlantGenomeAnnotations"
runBig(ws=wsthis, genome_id=genomeid)

contig_ids len 22
feat_data len 363
gene_3990
gene_3990
get_mrna_by_gene  2.17358088493 s
[u'mRNA_3889']
get_cds_by_mrna  2.24047613144 s
get_mrna_exons  2.02726984024 s
get_feature_locations  2.85375499725 s
get_feature_locations  2.74376106262 s
len(listmrna)
1
mrna mRNA_3889
gene_3991
gene_3991
get_mrna_by_gene  1.8807861805 s
[u'mRNA_3890']
get_cds_by_mrna  1.98944592476 s
get_mrna_exons  1.74452996254 s
get_feature_locations  2.15993118286 s
get_feature_locations  2.43378210068 s
len(listmrna)
1
mrna mRNA_3890
gene_3994
gene_3994
get_mrna_by_gene  2.11650681496 s
[u'mRNA_3893']
get_cds_by_mrna  1.99657201767 s
get_mrna_exons  2.04512000084 s
get_feature_locations  2.26067304611 s
get_feature_locations  2.58654904366 s
len(listmrna)
1
mrna mRNA_3893
gene_3995
gene_3995
get_mrna_by_gene  1.96572589874 s
[u'mRNA_3894']
get_cds_by_mrna  2.08969593048 s
get_mrna_exons  1.37279701233 s
get_feature_locations  2.15558600426 s
get_feature_locations  2.35059690475 s
len(listmrna)
1
mrna mRN

ConnectionError: HTTPSConnectionPool(host='ci.kbase.us', port=443): Max retries exceeded with url: /services/ws/ (Caused by <class 'socket.error'>: [Errno 50] Network is down)

In [ ]:
#write overall header
#for contigs
    #get contig features
    #write contig header
    #for gene in contig
        #write GFF gene
            #for mRNA from gene
                #write GFF mRNA for gene
                    #for exon in mRNA
                        #write GFF exon for mRNA
                        #write GFF CDS for mRNA
